In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("aggs").getOrCreate()

In [3]:
df = spark.read.csv("dataset/dataFrame/sales_info.csv", inferSchema=True, header=True)

In [4]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [5]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [21]:
df.groupBy("Company").sum().show() # sum(), min(), max(), count(), avg() etc

+-------+----------+
|Company|sum(Sales)|
+-------+----------+
|   APPL|    1480.0|
|   GOOG|     660.0|
|     FB|    1220.0|
|   MSFT|     967.0|
+-------+----------+



In [27]:
df.agg({"Sales": "max", "Company":"count"}).show()

+--------------+----------+
|count(Company)|max(Sales)|
+--------------+----------+
|            12|     870.0|
+--------------+----------+



In [29]:
df.describe().show()

+-------+-------+-------+------------------+
|summary|Company| Person|             Sales|
+-------+-------+-------+------------------+
|  count|     12|     12|                12|
|   mean|   null|   null| 360.5833333333333|
| stddev|   null|   null|250.08742410799007|
|    min|   APPL|  Chris|             120.0|
|    max|   MSFT|Vanessa|             870.0|
+-------+-------+-------+------------------+



In [33]:
# details steps
group_data = df.groupBy("Company")
group_data.agg({"Sales":"max"}).show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [40]:
# OR
df.groupby(["Company", "Person"]).max().show()

+-------+-------+----------+
|Company| Person|max(Sales)|
+-------+-------+----------+
|   GOOG|    Sam|     200.0|
|     FB|  Sarah|     350.0|
|   APPL|  Chris|     350.0|
|   MSFT|Vanessa|     243.0|
|   APPL|   John|     250.0|
|   APPL|  Linda|     130.0|
|   GOOG|Charlie|     120.0|
|   APPL|   Mike|     750.0|
|     FB|   Carl|     870.0|
|   MSFT|   Tina|     600.0|
|   MSFT|    Amy|     124.0|
|   GOOG|  Frank|     340.0|
+-------+-------+----------+



In [43]:
df.groupby(["Company", "Person"]).agg({"Sales":"min"}).show()

+-------+-------+----------+
|Company| Person|min(Sales)|
+-------+-------+----------+
|   GOOG|    Sam|     200.0|
|     FB|  Sarah|     350.0|
|   APPL|  Chris|     350.0|
|   MSFT|Vanessa|     243.0|
|   APPL|   John|     250.0|
|   APPL|  Linda|     130.0|
|   GOOG|Charlie|     120.0|
|   APPL|   Mike|     750.0|
|     FB|   Carl|     870.0|
|   MSFT|   Tina|     600.0|
|   MSFT|    Amy|     124.0|
|   GOOG|  Frank|     340.0|
+-------+-------+----------+



In [44]:
from pyspark.sql.functions import countDistinct, avg, stddev

In [51]:
df.select(countDistinct("Company")).show()

+-----------------------+
|count(DISTINCT Company)|
+-----------------------+
|                      4|
+-----------------------+



In [57]:
df.select(countDistinct("Company")).collect()[0].asDict()

{'count(DISTINCT Company)': 4}

In [60]:
# Aliasing
df.select(countDistinct("Company").alias("Company Count")).collect()[0].asDict()

{'Company Count': 4}

In [62]:
df.select(stddev("Sales").alias("Standard Deviation")).show()

+------------------+
|Standard Deviation|
+------------------+
|250.08742410799007|
+------------------+



In [63]:
from pyspark.sql.functions import format_number

In [81]:
sales_std = df.select(stddev("Sales").alias("std"))

In [82]:
sales_std.select(format_number("std", 2).alias("Std")).show()

+------+
|   Std|
+------+
|250.09|
+------+



In [83]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [87]:
# bydefault ascending order
df.orderBy("Sales").show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [89]:
# For descending order
df.orderBy(df.Sales.desc()).show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

